In [13]:
from tkinter.tix import Tree
import numpy as np

data = np.load("/root/myproject/GeneGraph/data/DrugComb/Process/cls/Drug_map.npy",allow_pickle=True)   # 读取 .npy 文件
print(type(data))                 # 看对象类型
print(data.shape)                 # 看数组形状
print(data.dtype)                 # 看数据类型
print(data)                       # 直接打印内容（小数组时可用）

import pickle

with open("KPGT_emb2304.pickle", "rb") as f:
    kpgt_data = pickle.load(f)

# print(kpgt_data[''])
for k,v in kpgt_data.items() :
    print(k,v)
    # break


<class 'numpy.ndarray'>
()
object
{'5-Fluorouracil': 0, 'etoposide': 1, 'mitomycin C': 2, '391210-10-9': 3, '7-Ethyl-10-hydroxycamptothecin': 4, 'Veliparib': 5, 'Vorinostat': 6, 'temozolomide': 7, '915019-65-7': 8, 'methotrexate': 9, 'Dasatinib': 10, 'Lapatinib': 11, 'Bortezomib': 12, 'paclitaxel': 13, 'Sorafenib': 14, 'Sunitinib': 15, 'Carboplatinum': 16, 'cyclophosphamide': 17, 'dexamethasone': 18, 'MK-1775': 19, 'Dinaciclib': 20, 'topotecan': 21, 'metformin': 22, 'Erlotinib': 23, 'MK-2206': 24, 'MK-5108': 25, 'Niraparib': 26, 'geldanamycin': 27, 'gemcitabine': 28, '891494-63-6': 29, 'Deforolimus': 30, 'vinblastine': 31, 'doxorubicin': 32, 'vinorelbine': 33, 'L-778123 free base': 34, 'MK-4541': 35, 'MRK003': 36, 'Eloxatin (TN)': 37}
BrC1C(Br)C(Br)C(Br)C(Br)C1Br [ 0.9660427  4.656059  -1.6086305 ...  2.2268302 -1.0574567 -1.6837476]
BrCC(=O)NCCc1c[nH]c2ccccc12 [ 1.0925436  2.2467766 -0.4672045 ...  0.7025719 -2.6921494  0.8734815]
BrCC(=O)NCCc1ccc2ccccc2c1 [ 2.5617952   3.7407708  -0.

In [5]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

def split(data, repeat):
    kf = KFold(n_splits=5, random_state=repeat, shuffle=True)
    seed = 42
    i = 1
    for train_index , test_index in kf.split(data):
        data_train_val = data.iloc[train_index]
        data_test = data.iloc[test_index]
        data_train, data_val = train_test_split(data_train_val, test_size=0.25, random_state=seed)
        all_index = np.concatenate((data_train.index.values, data_val.index.values, data_test.index.values), axis=0)
        all_index = np.unique(all_index)

        data_train = data_train.reset_index(drop=True)
        data_val = data_val.reset_index(drop=True)
        data_test = data_test.reset_index(drop=True)
        path_train = './data/repeat'+str(repeat)+'_fold'+str(i)+'_train.csv'
        path_val = './data/repeat'+str(repeat)+'_fold'+str(i)+'_val.csv'
        path_test = './data/repeat'+str(repeat)+'_fold'+str(i)+'_test.csv'
        i += 1
        print(path_train)
        data_train.to_csv(path_train)
        data_val.to_csv(path_val)
        data_test.to_csv(path_test)

data = pd.read_csv('./data_to_split.csv')
data = data.drop(columns='Unnamed: 0')
print(data)
for repeat in range(1,11):
    split(data, repeat=repeat)
    # break

      drug_row  drug_col  DepMap_ID  synergy_loewe  drug_pair  label
0            0        19         13      -9.013762          0      0
1            0         8         13      -7.869691          1      0
2            0        10         13       7.757410          2      1
3            0        20         13      -9.581096          3      0
4            0        23         13       9.637715          4      1
...        ...       ...        ...            ...        ...    ...
9636        33        25         14      -7.484658        540      0
9637        33         7         14     -16.089965        337      0
9638         6        24         14      10.806926        341      1
9639         6        37         14       5.709532        344      1
9640         6         7         14     -17.769911        347      0

[9641 rows x 6 columns]
./data/repeat1_fold1_train.csv
./data/repeat1_fold2_train.csv
./data/repeat1_fold3_train.csv
./data/repeat1_fold4_train.csv
./data/repeat1_fold5_tr

In [9]:
import os
import pickle
import numpy as np
import pandas as pd

def load_kpgt_dict(kpgt_pickle_path):
    with open(kpgt_pickle_path, "rb") as f:
        kpgt = pickle.load(f)   # dict: key(多半是SMILES) -> embedding
    # 统一成 np.float32 向量
    kpgt = {str(k).strip(): np.asarray(v, dtype=np.float32).ravel() for k, v in kpgt.items()}
    dims = {len(v) for v in kpgt.values()}
    if len(dims) != 1:
        raise ValueError(f"KPGT 维度不一致: {dims}")
    emb_dim = dims.pop()
    return kpgt, emb_dim

def map_kpgt_with_smiles_txt(
    kpgt_pickle_path: str,
    drug_map_npy_path: str,     # dict: drug_name -> index
    drug_smiles_txt_path: str,  # 两列: SMILES \t drug_name（无表头）
    output_prefix: str = "out/kpgt"
):
    os.makedirs(os.path.dirname(output_prefix), exist_ok=True)

    # 1) 读 KPGT（SMILES->向量，或可能是名字->向量）
    kpgt_dict, emb_dim = load_kpgt_dict(kpgt_pickle_path)
    kpgt_keys = set(kpgt_dict.keys())

    # 2) 读药名->序号
    name2idx = np.load(drug_map_npy_path, allow_pickle=True).item()
    name2idx = {str(k).strip(): int(v) for k, v in name2idx.items()}
    idx2name = {v: k for k, v in name2idx.items()}
    n_drug = len(name2idx)

    # 3) 读 drug_smile.txt（SMILES \t drug_name）
    # 若文件含表头, 把 header=None 改成 header=0 并设置 names=None
    df = pd.read_csv(drug_smiles_txt_path, sep=r"\t", header=None, names=["smiles","drug_name"], engine="python")
    # 去重/清洗
    df["smiles"] = df["smiles"].astype(str).str.strip()
    df["drug_name"] = df["drug_name"].astype(str).str.strip()
    df = df.dropna(subset=["smiles","drug_name"]).drop_duplicates(subset=["drug_name"])

    name2smiles = dict(zip(df["drug_name"], df["smiles"]))
    smiles2name = dict(zip(df["smiles"], df["drug_name"]))

    # 4) 判断 KPGT 的 key 是 SMILES 还是名字（以命中数为准）
    name_hits = len(set(name2idx.keys()) & kpgt_keys)
    smiles_hits = len(set(name2smiles.values()) & kpgt_keys)
    key_type = "smiles" if smiles_hits >= name_hits else "name"
    print(f"[INFO] key_type 判定: {key_type} (name_hits={name_hits}, smiles_hits={smiles_hits})")

    # 5) 构建对齐后的矩阵
    kpgt_mat = np.zeros((n_drug, emb_dim), dtype=np.float32)
    records = []
    filled = 0
    missing = []

    for idx in range(n_drug):
        name = idx2name[idx]
        smiles = name2smiles.get(name, None)

        vec, key_used = None, ""

        if key_type == "smiles":
            if smiles and smiles in kpgt_dict:
                vec, key_used = kpgt_dict[smiles], smiles
            elif name in kpgt_dict:
                vec, key_used = kpgt_dict[name], name
        else:  # key_type == "name"
            if name in kpgt_dict:
                vec, key_used = kpgt_dict[name], name
            elif smiles and smiles in kpgt_dict:
                vec, key_used = kpgt_dict[smiles], smiles

        if vec is not None:
            kpgt_mat[idx] = vec
            filled += 1
            has_emb = 1
        else:
            has_emb = 0
            missing.append((idx, name, smiles))

        records.append({
            "index": idx,
            "drug_name": name,
            "smiles": smiles if smiles else "",
            "has_embedding": has_emb,
            "key_used": key_used
        })

    npy_path = f"kgpt_matrix.npy"
    csv_path = f"kgpt_index_map.csv"
    np.save(npy_path, kpgt_mat)
    pd.DataFrame(records).to_csv(csv_path, index=False, encoding="utf-8")

    print(f"[DONE] 保存: {npy_path}")
    print(f"[DONE] 保存: {csv_path}")
    print(f"[STAT] 命中 {filled}/{n_drug} ({filled/n_drug:.1%})")
    if missing:
        print("[MISSING] 示例（前10条）:")
        for x in missing[:10]:
            print("  idx={}, name={}, smiles={}".format(*x))

    return kpgt_mat, records, missing


map_kpgt_with_smiles_txt(
    kpgt_pickle_path="KPGT_emb2304.pickle",
    drug_map_npy_path="Drug_map.npy",
    drug_smiles_txt_path="Drug_SMILE.txt",
    output_prefix="./"
)

[INFO] key_type 判定: smiles (name_hits=0, smiles_hits=0)
[DONE] 保存: kgpt_matrix.npy
[DONE] 保存: kgpt_index_map.csv
[STAT] 命中 0/38 (0.0%)
[MISSING] 示例（前10条）:
  idx=0, name=5-Fluorouracil, smiles=C1=C(C(=O)NC(=O)N1)F
  idx=1, name=etoposide, smiles=CC1OCC2C(O1)C(C(C(O2)OC3C4COC(=O)C4C(C5=CC6=C(C=C35)OCO6)C7=CC(=C(C(=C7)OC)O)OC)O)O
  idx=2, name=mitomycin C, smiles=CC1=C(C(=O)C2=C(C1=O)N3CC4C(C3(C2COC(=O)N)OC)N4)N
  idx=3, name=391210-10-9, smiles=C1=CC(=C(C=C1I)F)NC2=C(C=CC(=C2F)F)C(=O)NOCC(CO)O
  idx=4, name=7-Ethyl-10-hydroxycamptothecin, smiles=CCC1=C2C=C(C=CC2=NC3=C1CN4C3=CC5=C(C4=O)COC(=O)C5(CC)O)O
  idx=5, name=Veliparib, smiles=CC1(CCCN1)C2=NC3=C(C=CC=C3N2)C(=O)N
  idx=6, name=Vorinostat, smiles=C1=CC=C(C=C1)NC(=O)CCCCCCC(=O)NO
  idx=7, name=temozolomide, smiles=CN1C(=O)N2C=NC(=C2N=N1)C(=O)N
  idx=8, name=915019-65-7, smiles=CC(C)(C#N)C1=CC=C(C=C1)N2C3=C4C=C(C=CC4=NC=C3N(C2=O)C)C5=CC6=CC=CC=C6N=C5
  idx=9, name=methotrexate, smiles=CN(CC1=CN=C2C(=N1)C(=NC(=N2)N)N)C3=CC=C(C=C3)C(=O)N

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 [{'index': 0,
   'drug_name': '5-Fluorouracil',
   'smiles': 'C1=C(C(=O)NC(=O)N1)F',
   'has_embedding': 0,
   'key_used': ''},
  {'index': 1,
   'drug_name': 'etoposide',
   'smiles': 'CC1OCC2C(O1)C(C(C(O2)OC3C4COC(=O)C4C(C5=CC6=C(C=C35)OCO6)C7=CC(=C(C(=C7)OC)O)OC)O)O',
   'has_embedding': 0,
   'key_used': ''},
  {'index': 2,
   'drug_name': 'mitomycin C',
   'smiles': 'CC1=C(C(=O)C2=C(C1=O)N3CC4C(C3(C2COC(=O)N)OC)N4)N',
   'has_embedding': 0,
   'key_used': ''},
  {'index': 3,
   'drug_name': '391210-10-9',
   'smiles': 'C1=CC(=C(C=C1I)F)NC2=C(C=CC(=C2F)F)C(=O)NOCC(CO)O',
   'has_embedding': 0,
   'key_used': ''},
  {'index': 4,
   'drug_name': '7-Ethyl-10-hydroxycamptothecin',
   'smiles': 'CCC1=C2C=C(C=CC2=NC3=C1CN4C3=CC